# Creating the Dataset
This notebook shows some of the steps required in creating a dataset from a third party's
data. It has very little to do with DataRobot, and if you're mostly interested in learning
about how to use the DataRobot Python Client, then you could skip reading this section and miss
out on very little. In order to have the data necessary for the other notebook, you will
need to make sure that this notebook runs.

## What do I need to do?
### Get an API Key
The data we will be using is owned by the Federal Reserve Bank of St. Louis. They have an
API for which you will need a key. The key is free, don't worry. Grab one at
https://research.stlouisfed.org/docs/api/fred/

To run this notebook without any changes, you will need to save your API key in a file
in the same directory from which you run this notebook, and call the name of the file
`api_key`.

### Install the fredapi package
You will also need this python client package, which makes accessing the data incredibly
easy.

`pip install fredapi`

## What will we do with this data?
We're going to predict the future and get rich. 

More concretely, we're going to use historical economic data to build a forecasting
model for whether or not the US Economy will be in recession in 13 weeks from now.

## The FRED Economic Data
The Federal Reserve Bank of St. Louis provides a rich set of historical financial 
data, plus a REST API to access this data.

We have also written some utilities in order to make it easy to combine data 
series with different date frequencies in a technique known as Last Observation Carried Forward.

In [1]:
import warnings

import datetime
import fredapi
import pandas as pd
import timetools
fred = fredapi.Fred(api_key_file='api_key')

## Get the data
There is a lot of data accessible through the FRED API. More than a quarter million
data series, actually. That's probably too much to all be useful. 

We selected this set of series by starting with a subset of data specifically
related to the US Economy, and started filtering out forecast data, data that
was a pseudo-indicator date (which is a big data leak for this problem),
eventually ending up with the collection of series you see in the cell below.
It wasn't really a scientific process, there are certainly more robust ways
to go about it.

You can go learn about any of these on the FRED website, like this:
https://research.stlouisfed.org/fred2/series/A007RO1Q156NBEA
That is the webpage for the first data series in the cell below. You can
also get much of that data through the API, using the `get_series_info`
method like we do in the following cell.

In [2]:
good_columns = [
    u'A007RO1Q156NBEA', u'A011RE1Q156NBEA', u'A011RJ2Q224SBEA',
    u'A021RO1Q156NBEA', u'A021RY2Q224SBEA', u'A191RV1Q225SBEA',
    u'A765RL1Q225SBEA', u'A798RS2Q224SBEA', u'B808RA3Q086SBEA',
    u'CLSACBQ158SBOG', u'CORESTICKM158SFRBATL', u'DLTRUCKSSAAR',
    u'DNDGRY2Q224SBEA', u'DONGRS2Q224SBEA', u'DPCERV1Q225SBEA',
    u'DTRSRZ2Q224SBEA', u'LNS14024886', u'LNU02300000', u'LNU04000003',
    u'M1V', u'M2MOWN', u'M2V', u'MVAAUTOASS', u'NAPMNOI',
    u'NECDFNA066MNFRBPHI', u'NOCDSA156MSFRBPHI', u'PERMIT',
    u'PERMITMWNSA', u'PRS84006173', u'RCPHBS',
    u'STICKCPIXSHLTRM158SFRBATL', u'W004RZ2Q224SBEA', u'W087RA3Q086SBEA',
    u'W111RA3Q086SBEA', u'W117RL1Q225SBEA', u'W130RA3Q086SBEA',
    u'W368RG3Q066SBEA', u'WAAA', u'WGS10YR', u'WTB3MS',
    u'Y020RY2Q224SBEA', u'Y033RV1Q225SBEA', u'Y033RZ2Q224SBEA',
    u'Y034RA3Q086SBEA', u'Y034RY2Q224SBEA', u'Y052RL1Q225SBEA',
    u'Y054RG3Q086SBEA', u'Y060RZ2Q224SBEA', u'Y694RY2Q224SBEA']

## Get the metadata
We'll be needing to know the frequency of the observations in order
to merge the data correctly. That information is available from the
API. Each call to `get_series_info` involves an API call, so this may
take some time.

In [3]:
metadata = {}
for series_id in good_columns:
    try:
        metadata[series_id] = fred.get_series_info(series_id)
    except ValueError:
        # Series sometimes get retired from FRED
        warnings.warn('Series {} not found on FRED API'.format(series_id))

## Get the data
This is where we actually acquire the data. This next step may take a while.

In [4]:
def get_series_data(series_id):
    series_data = fred.get_series_first_release(series_id)

    series_index = [ix.strftime('%Y-%m-%d') for ix in series_data.index]
    series_data.index = series_index
    return series_data

obs = {}
for series_id in metadata.keys():
    series_data = get_series_data(series_id)
    obs[series_id] = series_data

## Organize by data frequency
Here we make a few groups of the series we just acquired. The ones that
have the same update frequency can be put into one dataframe very easily.

In [5]:
weekly = [series_id for series_id, meta
          in metadata.iteritems()
          if meta['frequency'] == 'Weekly, Ending Friday']
quarterly = [series_id for series_id, meta
             in metadata.iteritems()
             if meta['frequency'] == 'Quarterly']
monthly = [series_id for series_id, meta
           in metadata.iteritems()
           if meta['frequency'] == 'Monthly']

In [6]:
all_weekly = pd.DataFrame({metadata[series_id]['title']: obs[series_id]
                           for series_id in weekly})

all_monthly = pd.DataFrame({metadata[series_id]['title']: obs[series_id]
                            for series_id in monthly})

all_quarterly = pd.DataFrame({metadata[series_id]['title']: obs[series_id]
                              for series_id in quarterly})

## Combine the data of different frequencies
We wrote a little helper to take care of merging dataframes that have
differing date indexes. It comes in handy right here.

We also drop some rows that extend into the future - some of the series from
FRED come back like that, and it's not good for modeling.

In [7]:
fin_data = timetools.expand_frame_merge(all_weekly, all_monthly)
fin_data = timetools.expand_frame_merge(fin_data, all_quarterly)

fin_data = fin_data[fin_data.index <
                    datetime.datetime.today().strftime('%Y-%m-%d')]

## Create the target
The whole point of all this is to see if we can predict if there will be a
recession in the future, so we'll need to get historical data on the state
of the US economy.  

Of course, predicting if we are in a recession on any given day is kind of 
a no-brainer. So we'll slide the series in such a way that for any given
date, we're looking at whether there is a recession 13 weeks from that day.

In [8]:
usrec = fred.get_series_first_release('USREC')
usrec.index = [ix.isoformat().split('T')[0] for ix in usrec.index]
bool_match = usrec.index > '1918-01-01'
target_series = usrec[bool_match]


target_name = 'US Recession in 13 Weeks'
timetools.slide(target_series, 7 * 13)
target_frame = pd.DataFrame({target_name: target_series})


modeling_frame = timetools.expand_frame_merge(fin_data, target_frame)

## Trim some (mostly useless) data

Some of these series only started gathering data in the late 1940's. So we'll
just drop rows from before then, since there isn't much information in those
weeks. While this step isn't necessary, it does mean we'll be modeling on 
some more informative data.

In [9]:
na_counts = modeling_frame.isnull().sum(axis=1)
earliest_useful_day = na_counts[na_counts < 20].index[0]
earliest_useful_day

modeling_frame = modeling_frame[modeling_frame.index >= earliest_useful_day]


## Create the partition column
We'll be training on data before 1980, validating on data from 1980 to 1995, and withholding the data for 1995 onward. This is mostly arbitrary, but does ensure that each time interval has more than one recession. If we 
create a column with these labels, DataRobot will let us use that column to partition the data into 
training, validation, and holdout.

In [10]:
n_rows = len(modeling_frame)

validation_first_day = modeling_frame[modeling_frame.index >=
                                      '1980-01-01'].index[0]
validation_point = modeling_frame.index.get_loc(validation_first_day)
holdout_first_day = modeling_frame[modeling_frame.index >=
                                   '1995-01-01'].index[0]
holdout_point = modeling_frame.index.get_loc(holdout_first_day)

tvh = pd.Series(['T'] * n_rows)
tvh.loc[validation_point:holdout_point] = 'V'
tvh.loc[holdout_point:] = 'H'
tvh.index = modeling_frame.index

modeling_frame['TVH'] = tvh

Write the dataset to disk

In [11]:
fname = 'financials-{}.csv'.format(datetime.datetime.today().
                                   strftime('%Y-%m-%d'))
modeling_frame.to_csv(fname, index=True, index_label='Date', encoding='utf-8')